# Assignment 1
In this assignment you will be creating tools for learning and testing language models.
The corpora that you will be working with are lists of tweets in 8 different languages that use the Latin script. The data is provided either formatted as CSV or as JSON, for your convenience. The end goal is to write a set of tools that can detect the language of a given tweet.


*As a preparation for this task, download the data files from the course git repository.

The relevant files are under **lm-languages-data-new**:


*   en.csv (or the equivalent JSON file)
*   es.csv (or the equivalent JSON file)
*   fr.csv (or the equivalent JSON file)
*   in.csv (or the equivalent JSON file)
*   it.csv (or the equivalent JSON file)
*   nl.csv (or the equivalent JSON file)
*   pt.csv (or the equivalent JSON file)
*   tl.csv (or the equivalent JSON file)
*   test.csv (or the equivalent JSON file)





In [11]:
# imports
import pandas as pd
import numpy as np
from itertools import combinations
from collections import Counter
import time

In [12]:
!git clone https://github.com/kfirbar/nlp-course.git

fatal: destination path 'nlp-course' already exists and is not an empty directory.




---



**Important note: please use only the files under lm-languages-data-new and NOT under lm-languages-data**


---



In [13]:

!ls nlp-course/lm-languages-data-new


'ls' is not recognized as an internal or external command,
operable program or batch file.


In [14]:
data_files = {'en_df': 'en.csv',
              'es_df': 'es.csv',
              'fr_df': 'fr.csv',
              'in_df': 'in.csv',
              'it_df': 'it.csv',
              'nl_df': 'nl.csv',
              'pt_df': 'pt.csv',
              'tl_df': 'tl.csv'}
test_folder = r'C:\MSC\NLP2\nlp-course\lm-languages-data-new'
test_csv_files =  glob.glob(test_folder + '\\*.csv')
test_files =  {}
for i_file in test_csv_files:
    file_name_with_ending = os.path.basename(test_csv_files[0])
    file_name = os.path.splitext(file_name_with_ending)[0]
    test_files[file_name] = [i_file]

languages_list = list(data_files.key())
directory = 'nlp-course/lm-languages-data-new/'
start_token = '↠'
end_token = '↞'

NameError: name 'glob' is not defined

**Part 1**

Write a function *preprocess* that iterates over all the data files and creates a single vocabulary, containing all the tokens in the data. **Our token definition is a single UTF-8 encoded character**. So, the vocabulary list is a simple Python list of all the characters that you see at least once in the data.

In [ ]:
def preprocess(data_files):
    """
    data frame is table from 2 columns:
        1. tweet id
        2. tweet text
    """  
    tokens = []
    for path in data_files.values():
        df = pd.read_csv(path)
        if tokens.__len__() == 0 :
            columns_list = df.columns.to_list()
        for text in df[columns_list[-1]].values:
            tokens.extend(list(text))
    return list(set(tokens))

In [15]:
vocabulary = preprocess()

**Part 2**

Write a function lm that generates a language model from a textual corpus. The function should return a dictionary (representing a model) where the keys are all the relevant n-1 sequences, and the values are dictionaries with the n_th tokens and their corresponding probabilities to occur. For example, for a trigram model (tokens are characters), it should look something like:

{
  "ab":{"c":0.5, "b":0.25, "d":0.25},
  "ca":{"a":0.2, "b":0.7, "d":0.1}
}

which means for example that after the sequence "ab", there is a 0.5 chance that "c" will appear, 0.25 for "b" to appear and 0.25 for "d" to appear.

Note - You should think how to add the add_one smoothing information to the dictionary and implement it.

In [16]:
#helper function
def tweets_to_text(data_file_path, n):
    """
    data frame is table from 2 columns:
        1. tweet id
        2. tweet text
    """
    df = pd.read_csv(data_file_path)
    columns_list = df.columns.to_list()
    tweets_list = df[columns_list[-1]].apply(lambda x: start_token + x + end_token).values
    text = ''.join(tweets_list)
    text = start_token * (n-1) + text + end_token * (n-1)

    return text

In [17]:
def lm(n, vocabulary, data_file_path, add_one):
    # n - the n-gram to use (e.g., 1 - unigram, 2 - bigram, etc.)
    # vocabulary - the vocabulary list (which you should use for calculating add_one smoothing)
    # data_file_path - the data_file from which we record probabilities for our model
    # add_one - True/False (use add_one smoothing or not)
  
    lm_dict = {}
    V = len(vocabulary)

    text = tweets_to_text(data_file_path, n)

    # Extract n - 1 length substrings
    n_1_gram = [text[i: i + n-1] for i in range(len(text) - n-1)]
    counter_obj_n_1_gram = dict(Counter(n_1_gram))

    # Extract n length substrings
    n_gram = [text[i: i + n] for i in range(len(text) - n)]
    counter_obj_n_gram = dict(Counter(n_gram))

    for key in counter_obj_n_1_gram.keys():
        inner_dict = {}
        if add_one:
            gen = (key_1 for key_1 in counter_obj_n_gram.keys() if key_1[0:n-1] == key)
            for key_1 in gen:
                val = (int(counter_obj_n_gram[key_1]) + 1) / (int(counter_obj_n_1_gram[key]) + V)
                inner_dict[key_1[-1]] = val

            gen = (token for token in vocabulary if not(token in inner_dict))
            for key_1 in gen:
                val = 1 /  (int(counter_obj_n_1_gram[key]) + V)
                inner_dict[key_1[-1]] = val

        else:
            gen = (key_1 for key_1 in counter_obj_n_gram.keys() if key_1[0:n-1] == key)
            sum_vals = 0
            for key_1 in gen:
                val = int(counter_obj_n_gram[key_1]) / int(counter_obj_n_1_gram[key])
                inner_dict[key_1[-1]] = val
                sum_vals += val
            print(sum_vals)
            print(sum(list(inner_dict.values())))
        lm_dict[key] = inner_dict.copy()

    return lm_dict

In [18]:
test_dict = lm(1, vocabulary, directory + data_files['en_df'], False)

1.0000011566443356
1.0000011566443356


**Part 3**

Write a function *eval* that returns the perplexity of a model (dictionary) running over a given data file.

In [19]:
def eval(n, model, data_file):
    # n - the n-gram that you used to build your model (must be the same number)
    # model - the dictionary (model) to use for calculating perplexity
    # data_file - the tweets file that you wish to claculate a perplexity score for

    # read file
    text = tweets_to_text(data_file, n)

    # Extract n length substrings
    n_gram = [text[i: i + n] for i in range(len(text) - n)]

    model_keys = model.keys()
    entropy = 0 
    for i_letter in n_gram:
        if i_letter[0] in model_keys: 
            i_letter_model = model[i_letter[0]]
            if i_letter[1] in i_letter_model.keys():
                second_letter_prob = i_letter_model[i_letter[1]]
                entropy += -np.log2(second_letter_prob)
            else:
                entropy += 0

        else:
              entropy += 0
    entropy = entropy/n_gram.__len__()
    perplexity_score = 2**(entropy)
    return perplexity_score

**Part 4**

Write a function *match* that creates a model for every relevant language, using a specific value of *n* and *add_one*. Then, calculate the perplexity of all possible pairs (e.g., en model applied on the data files en ,es, fr, in, it, nl, pt, tl; es model applied on the data files en, es...). This function should return a pandas DataFrame with columns [en ,es, fr, in, it, nl, pt, tl] and every row should be labeled with one of the languages. Then, the values are the relevant perplexity values.

In [20]:

def match(n, add_one, data_files):
    # n - the n-gram to use for creating n-gram models
    # add_one - use add_one smoothing or not
    model_dict = {}
    result_dict = {}
    vocabulary = preprocess(data_files)
    for i_language_model in languages_list:
        
        i_model = lm(n, vocabulary, data_files[i_language_model], add_one)
        model_dict[i_language_model] = i_model
        result_dict[i_language_model] = {}

        for i_language_test in languages_list:
            i_language_model_i_score = eval(n, i_model, data_files[i_language_test])
            result_dict[i_language_model][i_language_test] = i_language_model_i_score
    print('summary for '+ i_language_model +' model perlexity score for each language:\n')
    perlexity_df = pd.DataFrame(result_dict)
    print(perlexity_df)
    #TODO
    return perlexity_df, model_dict

**Part 5**

Run match with *n* values 1-4, once with add_one and once without, and print the 8 tables to this notebook, one after another.

In [26]:

def run_match(data_files):
    match_dict = {}
    pr
    for n in range(2,5):
        match_dict[n] = {}
        add_one = True
        perlexity_df, model_dict = match(n, add_one, data_files)
        match_dict[n][add_one] = model_dict 
        add_one = False
        perlexity_df, model_dict = match(n, add_one, data_files)
        match_dict[n][add_one] = model_dict
    return match_dict

**Part 6**

Each line in the file test.csv contains a sentence and the language it belongs to. Write a function that uses your language models to classify the correct language of each sentence.

Important note regarding the grading of this section: this is an open question, where a different solution will yield different accuracy scores. any solution that is not trivial (e.g. returning 'en' in all cases) will be excepted. We do reserve the right to give bonus points to exceptionally good/creative solutions.

In [27]:
def classify(data_files):
    # TODO
    match_dict  = run_match(data_files)
    return match_dict
clasification_result = classify(data_files)

TypeError: preprocess() takes 0 positional arguments but 1 was given

**Part 7**

Calculate the F1 score of your output from part 6. (hint: you can use https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html). 


In [23]:
def calc_f1(result)
  # TODO

calc_f1(clasification_result)

SyntaxError: invalid syntax (Temp/ipykernel_29096/1993584275.py, line 1)

# **Good luck!**